In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd


In [ ]:
tweets_list1 = []

In [ ]:
sntwitter

In [ ]:
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Wegmans since:2020-03-01 until:2020-12-31').get_items()):
    if i>1000:
        break
    tweets_list1.append([tweet.date, tweet.id, tweet.content])

In [ ]:
tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Content'])


In [ ]:
tweets_df1.head()

In [ ]:
tweets_df1.dropna(inplace = True)

tweets_wegmans = tweets_df1[tweets_df1['Content'].str.contains("Wegmans")]

pd.set_option('display.max_colwidth', None)

tweets_wegmans



In [ ]:
!pip3 install -U emoji
import emoji
import regex

In [ ]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI["en"] for char in word):
            emoji_list.append(word)

    return emoji_list

In [ ]:
emoji_list = get_emojis(tweets_df1.Content[348])
print(' '.join(e for e in emoji_list))

In [ ]:
tweets_df1['emojis'] = tweets_df1['Content'].apply(get_emojis)

In [ ]:
display(tweets_df1.iloc[348])

In [ ]:
import re

In [ ]:
pat1 = r"http\S+"
pat2 = r"#"
pat3 = r"@"
pat4 = r"FAV"
pat5 = r"RE"
pat6 = r"pic.\S+"
pat7 = r"\n"
pat8 = '\r\n'
pat9 = r'|'.join((r'&amp;',r'&copy;',r'&reg;',r'&quot;',r'&gt;',r'&lt;',r'&nbsp;',r'&apos;',r'&cent;',r'&euro;',r'&pound;'))
combined_pat = r'|'.join((pat1, pat2, pat3, pat4, pat5, pat6, pat7, pat8, pat9))


In [ ]:
tweets_df1['stripped'] =  [re.sub(combined_pat, '', w) for w in tweets_df1.Content]
tweets_df1['stripped']  = [re.sub(r"  ", ' ', w) for w in tweets_df1.loc[:,'stripped']]
for i in range(340,352):
    print(tweets_df1.stripped[i])
    print('\n')

In [ ]:
for i in range(348):
    print(tweets_df1.stripped[i])
    print('\n')

In [ ]:
!pip install vaderSentiment 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
analyser = SentimentIntensityAnalyzer()

# Now get the compound sentiment score for each tweet
tweets_df1['C_Score'] = np.nan
for index, row in tweets_df1.iterrows(): 
    tweets_df1.loc[index, 'C_Score'] = analyser.polarity_scores(row['stripped'])['compound']
# Let's take a look!
pd.set_option('display.max_colwidth', None)
tweets_df1[['stripped','C_Score']][340:352]

In [ ]:
print(f"Count positive tweets: {sum(tweets_df1['C_Score'] > 0.05)}")
print(f"Count netural tweets: {tweets_df1['C_Score'].between(-0.05, 0.05).sum()}")
print(f"Count negative tweets: {sum(tweets_df1['C_Score'] < -0.05)}")
print(f"Total number of tweets: {tweets_df1['C_Score'].count()}")
      
display(tweets_df1.C_Score.describe())

In [ ]:
# import necessary modules (if not already imported)
import matplotlib.pyplot as plt
import seaborn as sns

# settings for seaborn plotting style
sns.set(color_codes=True)

# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(5,5)})

# Create Histogram
ax = sns.histplot(tweets_df1['C_Score'],
                  bins=10,
                  kde=False,
                  color='skyblue')
ax.set(xlabel='Sentiment Distribution', ylabel='Frequency')

In [ ]:
# create new column
tweets_df1['Sentiment'] = np.nan

# Loop through rows of dataframe and determine strings for new column "Sentiment"
for index, row in tweets_df1.iterrows(): 
    if tweets_df1.loc[index, 'C_Score'] > 0.05 : 
            tweets_df1.loc[index, 'Sentiment'] = "Positive"       
    elif tweets_df1.loc[index, 'C_Score'] < -0.05 :
            tweets_df1.loc[index, 'Sentiment'] = "Negative"   
    else : 
        tweets_df1.loc[index, 'Sentiment'] = "Neutral"

tweets_df1['Sentiment'] = tweets_df1['Sentiment'].astype("category")

In [ ]:
tweets_df1[['stripped','C_Score', 'Sentiment']].tail(25)

In [ ]:
# import necessary modules (if not already imported)
import matplotlib.pyplot as plt

# Set font size
plt.rcParams['font.size']=24

# Define figure
fig, ax = plt.subplots(figsize=(9, 6), subplot_kw=dict(aspect="equal"))

# Get count by sentiment category from tweets_df
sentiment_counts = tweets_df1.Sentiment.value_counts()
labels = sentiment_counts.index

# Define colors
color_palette_list = ['lightgreen', 'purple', 'red','orange']

# Generate graph components
wedges, texts, autotexts = ax.pie(sentiment_counts, wedgeprops=dict(width=0.5), startangle=-40,  
       colors=color_palette_list[0:3], autopct='%1.0f%%', pctdistance=.75, textprops={'color':"w", 'weight':'bold'})

# Plot wedges
for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    ax.annotate(labels[i], xy=(x, y), xytext=(1.2*x, 1.2*y),
                horizontalalignment=horizontalalignment)
# Set title
ax.set_title("Sentiment Distribution", y=.95, fontsize = 24)

# Show Donut Chart
plt.show()

In [ ]:
tweets = tweets_df1

In [ ]:
import math
tweets['day'] = [one.date() for one in tweets['Datetime']]
tweets = tweets.sort_values(by=['day'])

sentiments = ["Positive", "Neutral", "Negative"]
positiveProps = (tweets[tweets.Sentiment == 'Positive'].groupby(['day']).count()[['Sentiment']]/ tweets.groupby(['day']).count()[['Sentiment']])*100
neutralProps = (tweets[tweets.Sentiment == 'Neutral'].groupby(['day']).count()[['Sentiment']]/ tweets.groupby(['day']).count()[['Sentiment']])*100
negativeProps = (tweets[tweets.Sentiment == 'Negative'].groupby(['day']).count()[['Sentiment']]/ tweets.groupby(['day']).count()[['Sentiment']])*100
 
positiveProps = positiveProps['Sentiment'].tolist()
neutralProps = neutralProps['Sentiment'].tolist()
negativeProps = negativeProps['Sentiment'].tolist()
plt.figure(figsize=[16, 5])
barWidth = 0.5
labels = tweets.day.unique()
r = np.arange(len(labels))
positiveProps = [0 if math.isnan(x) else x for x in positiveProps]
neutralProps = [0 if math.isnan(x) else x for x in neutralProps]
negativeProps = [0 if math.isnan(x) else x for x in negativeProps]

plt.bar(r,positiveProps, color='lightgreen', edgecolor='white', width=barWidth)
plt.bar(r, neutralProps, bottom=positiveProps, color='skyblue', edgecolor='white', width=barWidth)
plt.bar(r, negativeProps, bottom=[i+j for i,j in zip(positiveProps, neutralProps)], color='orange', edgecolor='white', width=barWidth)
 
plt.xticks(r, labels, rotation = 45, fontsize=12)
plt.yticks(fontsize=16)
plt.suptitle('Sentiment Distribution over Time')
plt.xlabel("Date", fontsize=18)
plt.ylabel("Share", fontsize=20)
plt.legend(sentiments)
plt.show()
# Sort by Index again
tweets.sort_index(inplace=True)

In [ ]:
import numpy as np
import re

# Let's try to identify tweets that are about beer
tweets['Wegmans'] = np.where(tweets.stripped.str.contains('(?:^|\W)(wegmans|delivery|drinks|safety|shop)(?:$|\W)',                                                                                                      # beer|booze|brew|bottle|pint|hops|drink|beach
    flags = re.IGNORECASE), 1, 0)

# Let's try to identify tweets that are about the virus
tweets['Virus'] = np.where(tweets.stripped.str.contains('(?:^|\W)(virus|covid-19|SARS-COV-2|vaccine|cough|coronavirus|mask|sick|spread|pandemic|death|covid|hospital|n95|ventilator|respiratory)(?:$|\W)',                                                                                    # (virus|covid-19|SARS-COV-2|vaccine|cough|coronavirus|mask|sick|spread|pandemic|death|covid|hospital|n95|ventilator|respiratory)(?:$|\W)',
    flags = re.IGNORECASE), 1, 0)
 
# How many tweets of each topic?
print(f"Total {tweets['Wegmans'].count()}")
print(f"Wegmans {tweets['Wegmans'].sum()}")
print(f"Virus {tweets['Virus'].sum()}")

In [ ]:
pd.set_option('display.max_colwidth', None)
tweets[['stripped','Wegmans','Virus']].tail(30) 

In [ ]:
select_tweets = tweets.loc[tweets['Virus'] == 1, 'stripped'].values[:]
for w in select_tweets[3:13]:
    print(w)
    print('\n')

In [ ]:
select_tweets = tweets.loc[tweets['Wegmans'] == 1, 'stripped'].values[:]
for w in select_tweets[3:13]:
    print(w)
    print('\n')

In [ ]:
# 1. Import module 
from wordcloud import WordCloud

# 2. Create bag of words for tweets of certain sentiment
all_words = ' '.join([text for text in tweets[tweets['Sentiment'] == 'Positive']['stripped']])

# 3. Generate Word Cloud
wordcloud = WordCloud(collocations=True, width=800, height=500, random_state=5, max_font_size=110).generate(all_words)

# 4. Visulaize Cloud
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()